In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

import common
from search_tools import create_search_tools

evidently_search_tools = create_search_tools()

In [3]:
import inspect

search_tool = {
    "name": evidently_search_tools.search.__name__,
    "description": inspect.getdoc(evidently_search_tools.search),
    "input_schema": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "query parameter"
            }
        },
        "required": [
            "query"
        ]
    }
}

get_file_tool = {
    "name": evidently_search_tools.get_file.__name__,
    "description": inspect.getdoc(evidently_search_tools.get_file),
    "input_schema": {
        "type": "object",
        "properties": {
            "filename": {
                "type": "string",
                "description": "filename parameter"
            }
        },
        "required": [
            "filename"
        ]
    }
}

In [4]:
tool_pairs = [
    (evidently_search_tools.search, search_tool),
    (evidently_search_tools.get_file, get_file_tool)
]

In [5]:
tools = [t for (_, t) in tool_pairs]
tools_index = {t["name"]: f for (f, t) in tool_pairs}

In [6]:
def make_call(tool_call):
    name = tool_call.name

    if name in tools_index:
        func = tools_index[name]
        arguments = tool_call.input
        result = func(**arguments)
    else:
        result = f"Tool '{name}' not found in tools index"

    return {
        "type": "tool_result",
        "tool_use_id": tool_call.id,
        "content": json.dumps(result)
    }

In [30]:
from dataclasses import dataclass

@dataclass
class ToolCall:
    id: str
    name: str
    input: str

tool_call = ToolCall(name="search", input={"query": "dashboard"}, id="1234")
tool_call_result = make_call(tool_call)
tool_call_result

{'type': 'tool_result',
 'tool_use_id': '1234',
 'content': '[{"title": "Add dashboard panels (UI)", "description": "How to design your Dashboard with custom Panels.", "content": {"matches": ["**Dashboards** let you create Panels to...", "...ject.\\n\\n<Check>\\n  No-code **Dashboards** are available in the Evi...", "...anels appear on a single **Dashboard**. You can add multiple Ta..."], "total_matches": 11}, "filename": "docs/platform/dashboard_add_panels_ui.mdx"}, {"title": "Add dashboard panels (API)", "description": "How to design your Dashboard with custom Panels.", "content": {"matches": ["...oard_add_panels_ui).\\n\\n## **Dashboard** Management\\n\\n<Check>\\n  **Dashboards**...", "...rd Management\\n\\n<Check>\\n  **Dashboards** as code are available in...", "... Tab:\\n\\n```python\\nproject.**dashboard**.add_tab(\\"Another Tab\\")\\n`..."], "total_matches": 27}, "filename": "docs/platform/dashboard_add_panels.mdx"}, {"title": "Dashboard panel types [Legacy]", "description": 

In [7]:
from anthropic import Anthropic
anthropic_client = Anthropic()

In [32]:
messages = [
    {"role": "user", "content": common.DEFAULT_QUESTION}
]

# 1st call - initial

response = anthropic_client.messages.create(
    model="claude-haiku-4-5",
    max_tokens=4096,
    system=common.SINGLE_TOOL_SYSTEM_PROMPT,
    messages=messages,
    tools=tools,
)

In [33]:
response.content

[ToolUseBlock(id='toolu_01FMJcDzHPYnSkDGhqF8KnvH', input={'query': 'create dashboard'}, name='search', type='tool_use', caller={'type': 'direct'})]

In [34]:
tool_call = [m for m in response.content if m.type == 'tool_use'][0]
tool_call

ToolUseBlock(id='toolu_01FMJcDzHPYnSkDGhqF8KnvH', input={'query': 'create dashboard'}, name='search', type='tool_use', caller={'type': 'direct'})

In [35]:
print(f"Tool call: {tool_call.name}({tool_call.input})")

Tool call: search({'query': 'create dashboard'})


In [36]:
tool_call_result = make_call(tool_call)
tool_call_result

{'type': 'tool_result',
 'tool_use_id': 'toolu_01FMJcDzHPYnSkDGhqF8KnvH',
 'content': '[{"title": "Add dashboard panels (UI)", "description": "How to design your Dashboard with custom Panels.", "content": {"matches": ["**Dashboards** let you **create** Panels to...", "**Dashboards** let you **create** Panels to visualize eval...", "...ject.\\n\\n<Check>\\n  No-code **Dashboards** are available in the Evi..."], "total_matches": 14}, "filename": "docs/platform/dashboard_add_panels_ui.mdx"}, {"title": "Add dashboard panels (API)", "description": "How to design your Dashboard with custom Panels.", "content": {"matches": ["...oard_add_panels_ui).\\n\\n## **Dashboard** Management\\n\\n<Check>\\n  **Dashboards**...", "...rd Management\\n\\n<Check>\\n  **Dashboards** as code are available in...", "...(/docs/setup/cloud) and [**create** a Project](/docs/platfor..."], "total_matches": 32}, "filename": "docs/platform/dashboard_add_panels.mdx"}, {"title": "Custom Metric", "description": "How to cr

In [37]:
messages.append({"role": "assistant", "content": response.content})
messages.append({"role": "user", "content": [tool_call_result]})

In [38]:
# 2nd call

response = anthropic_client.messages.create(
    model="claude-haiku-4-5",
    max_tokens=4096,
    system=common.SINGLE_TOOL_SYSTEM_PROMPT,
    messages=messages,
    tools=tools,
)

In [39]:
response.content

[TextBlock(citations=None, text='Let me get more detailed information about creating dashboards:', type='text'),
 ToolUseBlock(id='toolu_0168ctyUXgGsoGNqFL4EBYzg', input={'filename': 'docs/platform/dashboard_overview.mdx'}, name='get_file', type='tool_use', caller={'type': 'direct'}),
 ToolUseBlock(id='toolu_01LAqBEd9kWwazZcxWMfQdNE', input={'filename': 'docs/platform/dashboard_add_panels_ui.mdx'}, name='get_file', type='tool_use', caller={'type': 'direct'})]

In [40]:
tool_call_results = []

for message in response.content:
    if message.type == 'tool_use':
        print(f'executing {message.name}({message.input})...')
        tool_call_output = make_call(message)
        tool_call_results.append(tool_call_output)

    if message.type == 'text':
        text = message.text
        print('ASSISTANT:', text)

ASSISTANT: Let me get more detailed information about creating dashboards:
executing get_file({'filename': 'docs/platform/dashboard_overview.mdx'})...
executing get_file({'filename': 'docs/platform/dashboard_add_panels_ui.mdx'})...


In [41]:
messages.append({"role": "assistant", "content": response.content})
messages.append({"role": "user", "content": tool_call_results})

In [42]:
# 3rd call - final

response = anthropic_client.messages.create(
    model="claude-haiku-4-5",
    max_tokens=4096,
    system=common.SINGLE_TOOL_SYSTEM_PROMPT,
    messages=messages,
    tools=tools,
)

In [43]:
response.content

[TextBlock(citations=None, text='Based on the documentation, here\'s how to create a dashboard in Evidently:\n\n## Basic Steps\n\n1. **Run an Evaluation and Save Reports**: First, you need to run an evaluation and save at least one Report to your Project. The Dashboard is empty until you have Reports with evaluation results.\n\n2. **Access the Dashboard**: Each Project has its own Dashboard that you can populate with Panels.\n\n## Creating Panels (UI Method)\n\nIf you\'re using **Evidently Cloud or Enterprise**, you can create Panels through the user interface:\n\n1. **Enter Edit Mode** - Click the edit button in the top right corner of the Dashboard\n2. **Click "Add Panel"** button\n3. **Configure the Panel** by:\n   - **Selecting a Metric** - Choose which metric to visualize (must match a metric name logged in your Reports)\n   - **Filtering by Tags** (optional) - Use the "From" field to filter data by specific Tags\n   - **Filtering by Metric Label** - For column-level metrics, use 

In [44]:
print(response.content[0].text)

Based on the documentation, here's how to create a dashboard in Evidently:

## Basic Steps

1. **Run an Evaluation and Save Reports**: First, you need to run an evaluation and save at least one Report to your Project. The Dashboard is empty until you have Reports with evaluation results.

2. **Access the Dashboard**: Each Project has its own Dashboard that you can populate with Panels.

## Creating Panels (UI Method)

If you're using **Evidently Cloud or Enterprise**, you can create Panels through the user interface:

1. **Enter Edit Mode** - Click the edit button in the top right corner of the Dashboard
2. **Click "Add Panel"** button
3. **Configure the Panel** by:
   - **Selecting a Metric** - Choose which metric to visualize (must match a metric name logged in your Reports)
   - **Filtering by Tags** (optional) - Use the "From" field to filter data by specific Tags
   - **Filtering by Metric Label** - For column-level metrics, use the "Where" selector to specify:
     - Column name


In [45]:
message_history = [
    {"role": "user", "content": common.DEFAULT_QUESTION}
]

iteration_number = 1

while True:

    response = anthropic_client.messages.create(
        model="claude-haiku-4-5",
        max_tokens=4096,
        system=common.SINGLE_TOOL_SYSTEM_PROMPT,
        messages=message_history,
        tools=tools,
    )
    
    print(f'iteration number {iteration_number}...')
    message_history.append({"role": "assistant", "content": response.content})

    tool_call_results = []

    for message in response.content:
        if message.type == 'tool_use':
            print(f'executing {message.name}({message.input})...')
            tool_call_output = make_call(message)
            tool_call_results.append(tool_call_output)
    
        if message.type == 'text':
            text = message.text
            print('ASSISTANT:', text)


    if len(tool_call_results) > 0:
        message_history.append({"role": "user", "content": tool_call_results})
    else:
        break

    iteration_number = iteration_number + 1
    print()


iteration number 1...
executing search({'query': 'create dashboard'})...

iteration number 2...
ASSISTANT: Let me get more detailed information about creating dashboards:
executing get_file({'filename': 'docs/platform/dashboard_overview.mdx'})...
executing get_file({'filename': 'docs/platform/dashboard_add_panels_ui.mdx'})...

iteration number 3...
ASSISTANT: ## How to Create a Dashboard in Evidently

Here are the key steps to create a dashboard:

### **1. Prerequisites**
- Each Project automatically has its own empty Dashboard
- To populate it, you must first **run an evaluation and save at least one Report** to the Project

### **2. Add Dashboard Tabs** (Optional)
- Enter "Edit" mode on the Dashboard (top right corner)
- Click the plus sign with "add Tab" on the left
- Choose either:
  - A **pre-built Tab** template (like "Columns" for text evaluations or column distributions)
  - An **empty Tab** with a custom name

### **3. Add Panels**
To add visualization Panels to your Dashboard

In [14]:
from agent_anthropic import AnthropicAgent

In [15]:
agent = AnthropicAgent(
    llm_client=anthropic_client,
    model="claude-haiku-4-5",
    instructions=common.SINGLE_TOOL_SYSTEM_PROMPT,
    tool_pairs=tool_pairs
)

In [16]:
messages = agent.loop(common.DEFAULT_QUESTION)

iteration number 1...
ASSISTANT: I'll search for information on how to create a dashboard in Evidently.
executing search({'query': 'create dashboard'})...

iteration number 2...
ASSISTANT: Let me get more detailed information from the dashboard overview and creation guides.
executing get_file({'filename': 'docs/platform/dashboard_overview.mdx'})...
executing get_file({'filename': 'docs/platform/dashboard_add_panels_ui.mdx'})...
executing get_file({'filename': 'docs/platform/dashboard_add_panels.mdx'})...

iteration number 3...
ASSISTANT: Based on the documentation, here's how to create a dashboard in Evidently:

## Overview

A **Dashboard** provides a clear view of your AI application performance. Each Project has its own Dashboard, which is empty at first. To populate it, you need to run an evaluation and **save at least one Report** to the Project.

## Basic Steps

1. **Create Reports**: First, run evaluations and save at least one Report to your Project. The Dashboard relies on Repo